# Advanced NLP Assignment 3, Group 1
Thijs Vollebregt(2670637), Chuqiao Guo(2798305), Yijing Zhang(2818171), Danna Shao(2663369)

### Importing libraries and utils:

In [2]:
import warnings
warnings.filterwarnings('ignore')

import transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_metric

from read_and_preprocess import *

### Basic settings
- We use seqeval as the metric, and bert-base-uncased as BERT model with its corresponding auto tokenizer

In [3]:
metric = load_metric("seqeval")
task = "srl"
model_checkpoint = "bert-base-uncased" # bert-base-uncased for better percision, distilbert-base-uncased for faster run
batch_size = 16
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

## 1. Baseline model
The appraoch used for the baseline model is basically converting the sentence into the following form:

> [CLS] This is the sentence content [SEP] is [SEP].

And this is realized by simply using the logic of the auto tokenizer:
`tokenizer(list1,list2)` will return [CLS] list1 content [SEP] list2 content [SEP].

### 1.1 Importing datasets and libraries:
Corpus is load and preprocessed into huggingface dataset type by baseline_ds.py. This script contains following functions and variables for the baseline model:

Dataset preparation:
- `get_mappings_dict()`: Get the dictionary mapping string classes (e.g. 'ARG0') to int labels and its reverse.
- `create_word_sentlist()`: Create dictionary containing the required data for generating desired huggingface dataset.
- `tokenize_and_align_labels()`: Adapted from the example notebook. Solves label alignment after re-tokenization.
    
Evaluation:
- `compute_metrics()`: Compute the overall percision, recall and f1.
- `reverse_label()`: Map the int class labels back to strings.
- `class_results()`: Compute the percision, recall and f1 for each class.

Variables: 
- `label_dict, label_dict_rev`: The mapping dictionary from string class to int class and its reverse.
- `tokenized_train, tokenized_dev, tokenized_test`: The tokenized and ready-to-use datasets.

In [4]:
import baseline_ds

Map:   0%|          | 0/42466 [00:00<?, ? examples/s]

Map:   0%|          | 0/5441 [00:00<?, ? examples/s]

Map:   0%|          | 0/5328 [00:00<?, ? examples/s]

Inspect the tokenized (ready to use) dataset:

In [10]:
print(baseline_ds.tokenized_test[0])

{'tokens': ['What', 'if', 'Google', 'Morphed', 'Into', 'GoogleOS', '?'], 'srl_arg_tags': [29, 29, 11, 29, 29, 43, 29], 'pred': ['Morphed'], 'input_ids': [101, 2054, 2065, 8224, 22822, 8458, 2098, 2046, 8224, 2891, 1029, 102, 22822, 8458, 2098, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'word_ids': [None, 0, 1, 2, 3, 3, 3, 4, 5, 5, 6, None, 0, 0, 0, None], 'labels': [-100, 29, 29, 11, 29, 29, 29, 29, 43, 43, 29, -100, 29, 29, 29, -100]}


We can see that the last 4 input ids are 102, x, x, 102 where 102 is the labels of [SEP], and their corresponding word_ids are None, indicating the tokenization worked properly.

### 1.2 Creating basline model:
The model is trained on the tokenized trian dataset, and evaluated on the dev dataset in the training process.

In [5]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(baseline_ds.label_dict))
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
label_list = baseline_ds.label_list

base_trainer = Trainer(
    model,
    args,
    train_dataset=baseline_ds.tokenized_train,
    eval_dataset=baseline_ds.tokenized_dev,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=baseline_ds.compute_metrics
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 1.3 Train and evaluate:
The evaluation is done on the test dataset

In [5]:
base_trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.088200,0.108037,0.798406,0.819914,0.809017,0.970571
2,0.063000,0.105916,0.817995,0.827277,0.822610,0.972739
3,0.048100,0.109497,0.820079,0.833860,0.826912,0.973297


TrainOutput(global_step=7965, training_loss=0.07123838856919187, metrics={'train_runtime': 352.3275, 'train_samples_per_second': 361.59, 'train_steps_per_second': 22.607, 'total_flos': 4555060725912480.0, 'train_loss': 0.07123838856919187, 'epoch': 3.0})

- Getting the prediction and true lables for the test dataset:

In [8]:
base_pred, base_labels, _ = base_trainer.predict(baseline_ds.tokenized_test)

- Evaluate the results of each class:

In [9]:
baseline_ds.class_results(base_pred, base_labels)

{'ADJ': {'precision': 0.7312252964426877,
  'recall': 0.7142857142857143,
  'f1': 0.72265625,
  'number': 259},
 'ADV': {'precision': 0.6910569105691057,
  'recall': 0.6378986866791745,
  'f1': 0.6634146341463414,
  'number': 533},
 'ARG0': {'precision': 0.9242424242424242,
  'recall': 0.8714285714285714,
  'f1': 0.8970588235294117,
  'number': 70},
 'ARG1': {'precision': 0.6554621848739496,
  'recall': 0.75,
  'f1': 0.6995515695067266,
  'number': 104},
 'ARG1-DSP': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'ARG2': {'precision': 0.2,
  'recall': 0.25,
  'f1': 0.22222222222222224,
  'number': 8},
 'ARG3': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2},
 'ARGM-ADJ': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'ARGM-ADV': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'ARGM-CXN': {'precision': 0.3333333333333333,
  'recall': 0.4,
  'f1': 0.3636363636363636,
  'number': 5},
 'ARGM-DIR': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 

### 1.4 Limitations of the baseline model:
- Cannot precisely handle sentences with duplicated predicates.
- 

## 2. More advanced model:
The more advanced model uses similar apporach as the Augment method described in [NegBERT (Khandelwal, et al. 2020)](http://www.lrec-conf.org/proceedings/lrec2020/pdf/2020.lrec-1.704.pdf). That is, adding a special token ([V]) immediately before the predicate:
> This [V] is a sentence.

(Note that **the special token and the predicate is considered a whole**. That is, the actual sentence is like
> 'This' **'[V] is'** 'a' 'sentence' '.'

### 2.1 Implementation
The implementation is actually simpler than the basic model, and the coding are only slightly adjusted.

We now use `augment_sentlist()` to create a list of sentences that having all the predicates augmented into '[V] pred', and removed the [SEP] part from the tokenizer.

In [8]:
import advanced_ds

Map:   0%|          | 0/42466 [00:00<?, ? examples/s]

Map:   0%|          | 0/5441 [00:00<?, ? examples/s]

Map:   0%|          | 0/5328 [00:00<?, ? examples/s]

Inspect the tokenized data again:

In [11]:
print(advanced_ds.tokenized_test[0])

{'tokens': ['What', 'if', 'Google', '[V] Morphed', 'Into', 'GoogleOS', '?'], 'srl_arg_tags': [29, 29, 11, 29, 29, 43, 29], 'input_ids': [101, 2054, 2065, 8224, 1031, 1058, 1033, 22822, 8458, 2098, 2046, 8224, 2891, 1029, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'word_ids': [None, 0, 1, 2, 3, 3, 3, 3, 3, 3, 4, 5, 5, 6, None], 'labels': [-100, 29, 29, 11, 29, 29, 29, 29, 29, 29, 29, 43, 43, 29, -100]}


The predicate is correctly augmented into '[V] pred'.

### 2.2 Creating the advanced model, train and evaluate:

In [12]:
advanced_trainer = Trainer(
    model,
    args,
    train_dataset=advanced_ds.tokenized_train,
    eval_dataset=advanced_ds.tokenized_dev,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=baseline_ds.compute_metrics
)

In [14]:
advanced_trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.044100,0.086001,0.851091,0.861808,0.856416,0.980564
2,0.028500,0.089585,0.855718,0.866338,0.860995,0.981127
3,0.018000,0.094870,0.860633,0.866519,0.863566,0.981430


TrainOutput(global_step=7965, training_loss=0.03031403552939588, metrics={'train_runtime': 339.2309, 'train_samples_per_second': 375.549, 'train_steps_per_second': 23.48, 'total_flos': 4617691475641200.0, 'train_loss': 0.03031403552939588, 'epoch': 3.0})

In [19]:
advanced_pred, advanced_labels, _ = advanced_trainer.predict(advanced_ds.tokenized_test)

- Evaluate the results of each class (slightly better than the baseline model):

In [20]:
baseline_ds.class_results(advanced_pred, advanced_labels)

{'ADJ': {'precision': 0.7361111111111112,
  'recall': 0.7130044843049327,
  'f1': 0.7243735763097949,
  'number': 223},
 'ADV': {'precision': 0.7036247334754797,
  'recall': 0.6707317073170732,
  'f1': 0.6867845993756504,
  'number': 492},
 'ARG0': {'precision': 0.9393939393939394,
  'recall': 0.8857142857142857,
  'f1': 0.9117647058823529,
  'number': 70},
 'ARG1': {'precision': 0.7788461538461539,
  'recall': 0.7788461538461539,
  'f1': 0.7788461538461539,
  'number': 104},
 'ARG1-DSP': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'ARG2': {'precision': 0.23529411764705882,
  'recall': 0.5,
  'f1': 0.31999999999999995,
  'number': 8},
 'ARG3': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2},
 'ARGM-ADJ': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'ARGM-ADV': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'ARGM-CXN': {'precision': 0.5714285714285714,
  'recall': 0.8,
  'f1': 0.6666666666666666,
  'number': 5},
 'ARGM-DIR': {'precis

# TODO
Descriptions and explanations (50%)
- [Done] Description of both models and their limitations (in particular with respect to the input representation)
- Description of the results
- Conclusions and directions for future research